In [29]:
import sagemaker
from sagemaker.huggingface import HuggingFaceModel
import boto3
import pandas as pd
import json
from sklearn.metrics import classification_report, confusion_matrix

# 1. 首先部署模型
def deploy_model():
    try:
        role = sagemaker.get_execution_role()
    except ValueError:
        iam = boto3.client('iam')
        role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']
    
    # 使用你训练好的模型创建HuggingFaceModel
    huggingface_model = HuggingFaceModel(
        model_data="s3://sagemaker-ap-northeast-1-034362076319/huggingface-pytorch-training-2024-11-08-15-54-35-976/output/model.tar.gz",
        role=role,
        transformers_version="4.37.0",
        pytorch_version="2.1.0",
        py_version='py310',
        entry_point='inference.py'
    )
    
    # 部署模型
    predictor = huggingface_model.deploy(
        initial_instance_count=1,
        instance_type='ml.m5.xlarge',
        endpoint_name='cold-sentiment-endpoint-3'
    )
    
    return predictor

# 2. 评估函数
def evaluate_model():
    # 加载eval数据集
    eval_data = pd.read_json('../eval_processed.jsonl', lines=True)
    
    # 创建预测客户端
    runtime_client = boto3.client('runtime.sagemaker')
    
    predictions = []
    probabilities_list = []  # 改名以更准确反映内容
    
    print("开始评估...")
    total = len(eval_data)
    
    for idx, row in eval_data.iterrows():
        if idx % 100 == 0:  # 每100条数据显示进度
            print(f"进度: {idx}/{total}")
            
        # 准备输入数据
        input_data = {"text": row['text']}
        print(f"{row['text']}")
        try:
            # 调用endpoint进行预测
            response = runtime_client.invoke_endpoint(
                EndpointName='cold-sentiment-endpoint-3',
                ContentType='application/json',
                Body=json.dumps(input_data)
            )
            
            # 解析结果
            result = json.loads(response['Body'].read().decode())
            print(f"预测结果 (index {idx}): {result['prediction']}--{row['label']}")

            predictions.append(result['prediction'])
            probabilities_list.append(result['probabilities'])  # 存储完整的概率分布
            
        except Exception as e:
            print(f"预测错误 (index {idx}): {str(e)}")
            predictions.append(None)
            probabilities_list.append(None)
    
    # 添加预测结果到数据框
    eval_data['predicted'] = predictions
    eval_data['probabilities'] = probabilities_list  # 存储完整的概率分布
    # 计算最高概率作为置信度
    eval_data['confidence'] = eval_data['probabilities'].apply(
        lambda x: max(x) if x is not None else None
    )
    
    # 输出评估结果
    print("\n=== 评估结果 ===")
    print("\n分类报告:")
    print(classification_report(eval_data['label'], eval_data['predicted']))
    
    print("\n混淆矩阵:")
    cm = confusion_matrix(eval_data['label'], eval_data['predicted'])
    print(cm)
    
    # 计算额外统计信息
    eval_data['is_correct'] = eval_data['label'] == eval_data['predicted']
    print("\n模型性能统计:")
    print(f"总样本数: {len(eval_data)}")
    print(f"准确率: {eval_data['is_correct'].mean():.4f}")
    print(f"平均置信度: {eval_data['confidence'].mean():.4f}")
    
    # 错误分析
    print("\n错误预测分析:")
    errors = eval_data[~eval_data['is_correct']].sort_values('confidence', ascending=False)
    print("\n高置信度错误案例:")
    for _, row in errors.head().iterrows():
        print(f"\n文本: {row['text']}")
        print(f"真实标签: {row['label']}")
        print(f"预测标签: {row['predicted']}")
        print(f"置信度: {row['confidence']:.4f}")
    
    # 保存评估结果
    eval_data.to_csv('model_evaluation_results-3.csv', index=False)
    return eval_data

# 3. 清理资源函数
def cleanup_endpoint():
    client = boto3.client('sagemaker')
    try:
        client.delete_endpoint(EndpointName='cold-sentiment-endpoint')
        print("已删除endpoint")
    except Exception as e:
        print(f"删除endpoint时出错: {str(e)}")

if __name__ == "__main__":
    try:
        # # 1. 部署模型
        print("正在部署模型...")
        predictor = deploy_model()
        
        # 2. 进行评估
        print("开始评估...")
        results = evaluate_model()
        
        # # 3. 完成后清理资源
        # print("评估完成，正在清理资源...")
        # cleanup_endpoint()
        
        print("评估结果已保存到 model_evaluation_results-3.csv")
        
    except Exception as e:
        print(f"发生错误: {str(e)}")
        # 确保即使发生错误也清理资源
        # cleanup_endpoint()

正在部署模型...
-------!开始评估...
开始评估...
进度: 0/2856
Rehabilitation training enhances functional levels. Rehabilitation training: rehabilitation training; functional levels: functional levels 
预测结果 (index 0): 0--0
永远做你们的贴心小棉袄 
预测结果 (index 1): 0--0
评论厨子做的菜好不好吃我还要先会做饭吗？
预测结果 (index 2): 1--1
我真的很想帮你治疗，但我是一名家庭医生，而你是一个孤儿。啊
预测结果 (index 3): 0--1
客户服务热情贴心提升形象 
预测结果 (index 4): 0--0
太仔细看看我这巴掌真适合你那大脸
预测结果 (index 5): 1--1
记得-下点-调料-，-这样-你家-人-吃-起来-才-会-有-味道
预测结果 (index 6): 0--1
我真的很想帮你治疗，但我是一名家庭医生，而你是一个孤儿。了
预测结果 (index 7): 0--1
就是嫉妒我户口本可翻页（说他是孤儿）
预测结果 (index 8): 1--1
我真的很想帮你治疗，但但我是一名家庭医生，而你是一个孤儿。
预测结果 (index 9): 0--1
好坏淫
预测结果 (index 10): 1--1
医学研究应关注不同人群的健康需求和差异 
预测结果 (index 11): 0--0
好特爱
预测结果 (index 12): 0--1
被分手吧
预测结果 (index 13): 1--1
二。大爷
预测结果 (index 14): 1--1
傻啊
预测结果 (index 15): 1--1
特别怂货
预测结果 (index 16): 1--1
艰不拆吧
预测结果 (index 17): 1--1
就是嫉我户口本可以翻页（说他是孤儿）
预测结果 (index 18): 1--1
你.看上去.好.眼熟.像.我.昨天.扔出去.的.那袋.垃圾
预测结果 (index 19): 1--1
记得|下点|调料|，|这样|你家|人|吃|起来|才|会|有|味道
预测结果 (index 20): 0--1
我们#要#向前#看#,#不错#过些#歪#瓜劣

In [ ]:
!source ../ven